In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#tf.debugging.set_log_device_placement(True)

# # Create some tensors
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)

# print(c)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from PIL import Image

In [5]:
def create_class_weights(dataset,num_classes):
    class_weights = {}
    for i in range(dataset.shape[0]):
        for j in range(num_classes):
            if dataset.iloc[i][j]==1:
                if j in class_weights:
                    class_weights[j]+=1
                else:
                    class_weights[j]=1

    x=max(class_weights,key=class_weights.get)
    z=class_weights[x]
    
    for key in class_weights:
        class_weights[key]=int(z/class_weights[key])
    classes={}
    for j in range(num_classes):
        if dataset.columns.values[j] not in classes:
            classes[dataset.columns.values[j]] = j
    return classes,class_weights


In [6]:
gender_dataset = pd.read_csv('gender.csv').drop(columns = 'Unnamed: 0').set_index("JPG")
gender_dataset
gender_classes, gender_class_weights =create_class_weights(gender_dataset,2)

In [7]:
age_df = pd.read_csv("age.csv").drop(columns = 'Unnamed: 0').set_index("JPG")
age_df
age_classes, age_class_weights = create_class_weights(age_df,8)

In [8]:
ethnicity_df = pd.read_csv("ethnicity.csv").drop(columns = 'Unnamed: 0').set_index("JPG")
ethnicity_df
ethnicity_classes, ethnicity_class_weights = create_class_weights(ethnicity_df,5)

In [10]:
# file_names = os.listdir('crop_part1')


In [41]:
# import re
# pattern = re.compile(r"(\d+)_(\d+)_(\d+)")

# df = pd.DataFrame(columns = ['Age', 'Gender', 'Ethnicity'])
# count = 0
# for i in file_names:
#     data =list(pattern.search(i).groups())
#     df.loc[count] = data
#     count += 1
        
    
# #7881 and 7876 are bad values
# # AGE, Gender, Ethnicity
# # 0 is male, 1 is female
# # Race is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).
# # 0 = White
# # 1 = Black
# # 2 = Asian
# # 3 = Indian
# # 4 = Others

# df['JPG'] = file_names
# img_path = []
# count = 0
# for row in df.iterrows():
#     path = "crop_part1/"+df.JPG.iloc[count]
#     img_path.append(path)
#     count += 1
# df['img_path'] = img_path
# df2 = df[['Age', 'Gender', 'Ethnicity', 'JPG', 'img_path']].copy()
# df2.head()

In [9]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [10]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# for layer in vgg16_conv.layers:
#     layer.trainable = False


In [11]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
vgg16_conv = add_regularization(vgg16_conv, reg)

In [12]:
#Check of regularization was added!
vgg16_conv.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0073838485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.006740606>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.007655664>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008172592>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008904765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0089925155>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00949263>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0119424565>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013766433>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015087408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017912947>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018236415>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017235568>]

In [13]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
#vgg_feedforward = Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(vgg_feedforward)
prediction = Dense(2, activation='softmax')(vgg_feedforward)

model_gender = Model(inputs=vgg16_conv.input, outputs=prediction)

In [14]:
model_gender.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
model_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

gender_fit = model_gender.fit(
  train,
  validation_data=val,
  epochs=3,
  steps_per_epoch = 14224//16,
  validation_steps = 4741//16,
)

Epoch 1/3
889/889 [==============================] - 156s 167ms/step - loss: 0.6557 - accuracy: 0.8262 - val_loss: 0.4703 - val_accuracy: 0.8661
Epoch 2/3
889/889 [==============================] - 274s 308ms/step - loss: 0.4297 - accuracy: 0.8826 - val_loss: 0.4460 - val_accuracy: 0.8792accuracy: 
Epoch 3/3
889/889 [==============================] - 184s 206ms/step - loss: 0.3849 - accuracy: 0.8998 - val_loss: 0.4153 - val_accuracy: 0.8896


In [16]:
# print("Evaluate on training data")
# results = model_gender.evaluate(train)
# print("train loss, train acc:", results)
# print("Evaluate on test data")
# results = model_gender.evaluate(test)
# print("test loss, test acc:", results)

# Gender w/o Regularization

In [17]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Gender_Images2/Train'
validation_folder = 'Gender_Images2/Val'
test_folder='Gender_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "binary", color_mode="rgb", classes = gender_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size=16, class_mode= "binary", color_mode="rgb",classes = gender_classes)

Found 14224 images belonging to 2 classes.
Found 4741 images belonging to 2 classes.
Found 4743 images belonging to 2 classes.


In [18]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [19]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
#vgg_feedforward = Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(vgg_feedforward)
prediction = Dense(2, activation='softmax')(vgg_feedforward)

model_gender = Model(inputs=vgg16_conv.input, outputs=prediction)

In [20]:
model_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

gender_fit = model_gender.fit(
  train,
  validation_data=val,
  epochs=3,
  steps_per_epoch = 14224//16,
  validation_steps = 4741//16,
)

Epoch 1/3
889/889 [==============================] - 219s 245ms/step - loss: 0.4936 - accuracy: 0.8151 - val_loss: 0.3011 - val_accuracy: 0.8729
Epoch 2/3
889/889 [==============================] - 260s 293ms/step - loss: 0.2764 - accuracy: 0.8838 - val_loss: 0.3438 - val_accuracy: 0.8666
Epoch 3/3
889/889 [==============================] - 295s 332ms/step - loss: 0.2251 - accuracy: 0.9062 - val_loss: 0.2918 - val_accuracy: 0.8902


# Resnet50

In [10]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [11]:
ResNet = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in ResNet.layers:
    layer.trainable = False
    
x = Flatten()(ResNet.output)

In [12]:
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(2, activation='softmax')(x)
model_ResNet_gender = Model(inputs= ResNet.input, outputs=prediction)
model_ResNet_gender.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [14]:
# num_steps_1 =13206//16,
# num_steps_2 = 4403//16
model_ResNet_gender.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']#,run_eagerly=True
)

model_ResNet_fit = model_ResNet_gender.fit(
    train,
    validation_data=val, 
    epochs=5, 
    steps_per_epoch=14224//32,
  validation_steps = 4741//32
)

Epoch 1/5
444/444 [==============================] - 127s 246ms/step - loss: 1.0921 - accuracy: 0.8453 - val_loss: 0.3017 - val_accuracy: 0.8661
Epoch 2/5
444/444 [==============================] - 77s 173ms/step - loss: 0.1804 - accuracy: 0.9295 - val_loss: 0.3597 - val_accuracy: 0.8708
Epoch 3/5
444/444 [==============================] - 76s 170ms/step - loss: 0.1399 - accuracy: 0.9579 - val_loss: 0.3865 - val_accuracy: 0.8714
Epoch 4/5
444/444 [==============================] - 140s 316ms/step - loss: 0.0607 - accuracy: 0.9779 - val_loss: 0.5478 - val_accuracy: 0.8801
Epoch 5/5
141/444 [========>.....................] - ETA: 34s - loss: 0.0448 - accuracy: 0.9858

KeyboardInterrupt: 

# Age

In [21]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

Found 14223 images belonging to 8 classes.
Found 4742 images belonging to 8 classes.
Found 4743 images belonging to 8 classes.


In [22]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg16_conv.trainable = True

In [23]:
#Add regularization

import tempfile

def add_regularization(model, regularizer):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

reg = tf.keras.regularizers.l2(0.0001)
vgg16_conv = add_regularization(vgg16_conv, reg)

In [24]:
#Check of regularization was added!
vgg16_conv.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0073838485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.006740606>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.007655664>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008172592>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008904765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0089925155>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00949263>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0119424565>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.013766433>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015087408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017912947>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.018236415>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017235568>]

In [ ]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)

prediction = Dense(8, activation='softmax')(vgg_feedforward)


model_age = Model(inputs=vgg16_conv.input, outputs=prediction)

In [ ]:
model_age.summary()

In [ ]:
model_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss')

vgg16_fit = model_age.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 14224//8,
  validation_steps = 4741//8
  
)#callbacks=[es]

# Age w/o Regularization

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Age_Images2/Train'
validation_folder = 'Age_Images2/Val'
test_folder='Age_Images2/Test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb", classes = age_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 8, class_mode= "sparse", color_mode="rgb",classes = age_classes)

In [ ]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg16_conv.layers:
    layer.trainable = False
    
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)

prediction = Dense(8, activation='softmax')(vgg_feedforward)


model_age = Model(inputs=vgg16_conv.input, outputs=prediction)

In [ ]:
model_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss')

vgg16_fit = model_age.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 14224//8,
  validation_steps = 4741//8
  
)#callbacks=[es]

In [14]:
print("Evaluate on training data")
results = model_age.evaluate(train)
print("train loss, train acc:", results)
print("Evaluate on test data")
results = model_age.evaluate(test)
print("test loss, test acc:", results)

Evaluate on training data
1778/1778 [==============================] - 102s 57ms/step - loss: 0.5919 - accuracy: 0.7472
train loss, train acc: [0.591942548751831, 0.7472403645515442]
Evaluate on test data
593/593 [==============================] - 25s 41ms/step - loss: 1.2983 - accuracy: 0.5745
test loss, test acc: [1.2982898950576782, 0.5745308995246887]


# ResNet50

In [10]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [11]:
ResNet = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in ResNet.layers:
    layer.trainable = False
    
x = Flatten()(ResNet.output)

In [12]:
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet_age = Model(inputs= ResNet.input, outputs=prediction)
model_ResNet_age.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
# num_steps_1 =13206//16,
# num_steps_2 = 4403//16
model_ResNet_age.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']#,run_eagerly=True
)

model_ResNet_fit = model_ResNet_age.fit(
    train,
    validation_data=val, 
    epochs=5, 
    steps_per_epoch=14224//8,
  validation_steps = 4741//8
)

Epoch 1/5
1778/1778 [==============================] - 189s 95ms/step - loss: 1.8030 - accuracy: 0.5230 - val_loss: 1.1240 - val_accuracy: 0.5608
Epoch 2/5
1778/1778 [==============================] - 143s 80ms/step - loss: 0.9636 - accuracy: 0.6052 - val_loss: 1.2926 - val_accuracy: 0.5481
Epoch 3/5
1778/1778 [==============================] - 226s 127ms/step - loss: 0.7811 - accuracy: 0.6770 - val_loss: 1.2974 - val_accuracy: 0.5621
Epoch 4/5
1778/1778 [==============================] - 168s 95ms/step - loss: 0.6449 - accuracy: 0.7429 - val_loss: 1.4546 - val_accuracy: 0.5766s
Epoch 5/5
 893/1778 [==============>...............] - ETA: 58s - loss: 0.4890 - accuracy: 0.8103

## Ethnicity

In [15]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam, SGD

training_folder='Ethnicity_Images/train'
validation_folder = 'Ethnicity_Images/val'
test_folder='Ethnicity_Images/test'

train_generator = ImageDataGenerator(rescale=1/255)
train = train_generator.flow_from_directory(training_folder, target_size=(224,224), 
                                              batch_size=16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

val_generator = ImageDataGenerator(rescale=1/255)
val = val_generator.flow_from_directory(validation_folder, target_size=(224,224), 
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb", classes = ethnicity_classes)


test_generator = ImageDataGenerator(rescale=1/255)
test = test_generator.flow_from_directory(test_folder, target_size=(224,224),
                                              batch_size = 16, class_mode= "sparse", color_mode="rgb",classes = ethnicity_classes)

Found 13206 images belonging to 5 classes.
Found 4403 images belonging to 5 classes.
Found 4404 images belonging to 5 classes.


In [16]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
vgg16_conv = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg16_conv.layers:
    layer.trainable = False

In [17]:
vgg_feedforward = Flatten()(vgg16_conv.output)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
vgg_feedforward = Dense(4096,activation = 'relu')(vgg_feedforward)
prediction = Dense(5, activation='softmax')(vgg_feedforward)

model_ethnicity = Model(inputs=vgg16_conv.input, outputs=prediction)

In [18]:
model_ethnicity.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
model_ethnicity.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

vgg16_fit = model_ethnicity.fit(
  train,
  validation_data=val,
  epochs=4,
  steps_per_epoch = 13206//16,
  validation_steps = 4403//16,
  callbacks=[es]
)

Epoch 1/4
825/825 [==============================] - 149s 176ms/step - loss: 1.0090 - accuracy: 0.6657 - val_loss: 0.6676 - val_accuracy: 0.7448
Epoch 2/4
825/825 [==============================] - 108s 131ms/step - loss: 0.5761 - accuracy: 0.7826 - val_loss: 0.6580 - val_accuracy: 0.7500
Epoch 3/4
825/825 [==============================] - 121s 147ms/step - loss: 0.4663 - accuracy: 0.8267 - val_loss: 0.6245 - val_accuracy: 0.7802
Epoch 4/4
825/825 [==============================] - 147s 178ms/step - loss: 0.3979 - accuracy: 0.8510 - val_loss: 0.8040 - val_accuracy: 0.7436


In [25]:
# predictions = model_ethnicity.predict_generator(test,steps =  4403//16)
# predicted_classes = [1 * (x[0]>=0.5) for x in predictions]

# 2.Get ground-truth classes and class-labels
true_classes = test.classes
class_labels = list(test.class_indices.keys()) 

# 3. Use scikit-learn to get statistics
from sklearn.metrics import confusion_matrix,classification_report

print(class_labels)

print(confusion_matrix(test.classes[:4400], predicted_classes))


['White', 'Black', 'Asian', 'Indian', 'Other']
[[ 962 1054    0    0]
 [ 436  470    0    0]
 [ 346  341    0    0]
 [ 351  440    0    0]]


In [37]:
print("Evaluate on training data")
results = model_ethnicity.evaluate(train)
print("train loss, train acc:", results)
print("Evaluate on test data")
results = model_ethnicity.evaluate(test)
print("test loss, test acc:", results)

Evaluate on training data
826/826 [==============================] - 81s 98ms/step - loss: 0.2005 - accuracy: 0.9269
train loss, train acc: [0.20052257180213928, 0.9269271492958069]
Evaluate on test data
276/276 [==============================] - 48s 174ms/step - loss: 0.8648 - accuracy: 0.7811
test loss, test acc: [0.8648311495780945, 0.7811080813407898]


In [ ]:
print("Evaluate on training data")
results = model_ResNet.predict_generator(test_data_generator,steps = 4403//16 )

# ResNet50

In [9]:
IMAGE_SIZE = [224, 224]
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [10]:
ResNet = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in ResNet.layers:
    layer.trainable = False
    
x = Flatten()(ResNet.output)

In [11]:
x = Dense(1024,activation = 'relu')(x)
prediction = Dense(8, activation='softmax')(x)
model_ResNet = Model(inputs= ResNet.input, outputs=prediction)
model_ResNet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [16]:
# num_steps_1 =13206//16,
# num_steps_2 = 4403//16
model_ResNet.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']#,run_eagerly=True
)

model_ResNet_fit = model_ResNet.fit(
    train,
    validation_data=val, 
    epochs=5, 
    steps_per_epoch=13206//16,
    validation_steps=4403//16)

Epoch 1/5
825/825 [==============================] - 106s 118ms/step - loss: 0.5369 - accuracy: 0.8281 - val_loss: 0.6910 - val_accuracy: 0.7807
Epoch 2/5
825/825 [==============================] - 85s 103ms/step - loss: 0.2437 - accuracy: 0.9206 - val_loss: 0.8373 - val_accuracy: 0.7770
Epoch 3/5
825/825 [==============================] - 121s 147ms/step - loss: 0.1818 - accuracy: 0.9393 - val_loss: 1.2632 - val_accuracy: 0.7675
Epoch 4/5
825/825 [==============================] - 127s 154ms/step - loss: 0.1112 - accuracy: 0.9646 - val_loss: 1.3907 - val_accuracy: 0.7716
Epoch 5/5
825/825 [==============================] - 90s 109ms/step - loss: 0.0955 - accuracy: 0.9682 - val_loss: 1.2963 - val_accuracy: 0.7818


In [19]:
print("Evaluate on training data")
results = model_ResNet.predict_generator(test_data_generator,steps = 4403//16 )


Evaluate on training data


NameError: name 'test_data_generator' is not defined